In [1]:
import pandas as pd
from collections import Counter
from sklearn.preprocessing import LabelEncoder
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import  InputLayer
from tensorflow.keras.layers import  Dense


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


2024-03-30 19:24:51.783367: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [24]:
# Model set up, tuning and model metrics packages.
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

In [4]:
#Load the data
df = pd.read_json('transactions.txt', lines=True)
df.head()

,accountNumber,customerId,creditLimit,availableMoney,transactionDateTime,transactionAmount,merchantName,acqCountry,merchantCountryCode,posEntryMode,...,echoBuffer,currentBalance,merchantCity,merchantState,merchantZip,cardPresent,posOnPremises,recurringAuthInd,expirationDateKeyInMatch,isFraud
0,737265056,737265056,5000,5000.0,2016-08-13T14:27:32,98.55,Uber,US,US,02,...,,0.0,,,,False,,,False,False
1,737265056,737265056,5000,5000.0,2016-10-11T05:05:54,74.51,AMC #191138,US,US,09,...,,0.0,,,,True,,,False,False
2,737265056,737265056,5000,5000.0,2016-11-08T09:18:39,7.47,Play Store,US,US,09,...,,0.0,,,,False,,,False,False
3,737265056,737265056,5000,5000.0,2016-12-10T02:14:50,7.47,Play Store,US,US,09,...,,0.0,,,,False,,,False,False
4,830329091,830329091,5000,5000.0,2016-03-24T21:04:46,71.18,Tim Hortons #947751,US,US,02,...,,0.0,,,,True,,,False,False


In [5]:
#print columns and their respective types
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 786363 entries, 0 to 786362
Data columns (total 29 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   accountNumber             786363 non-null  int64  
 1   customerId                786363 non-null  int64  
 2   creditLimit               786363 non-null  int64  
 3   availableMoney            786363 non-null  float64
 4   transactionDateTime       786363 non-null  object 
 5   transactionAmount         786363 non-null  float64
 6   merchantName              786363 non-null  object 
 7   acqCountry                786363 non-null  object 
 8   merchantCountryCode       786363 non-null  object 
 9   posEntryMode              786363 non-null  object 
 10  posConditionCode          786363 non-null  object 
 11  merchantCategoryCode      786363 non-null  object 
 12  currentExpDate            786363 non-null  object 
 13  accountOpenDate           786363 non-null  o

In [6]:
print(Counter(df["isFraud"]))

Counter({False: 773946, True: 12417})


In [22]:
df.dropna(how='all', axis=1, inplace=True) 

In [8]:
# drop rows with NaN value
df = df.dropna()

In [9]:
#print columns and their respective types
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 786363 entries, 0 to 786362
Data columns (total 29 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   accountNumber             786363 non-null  int64  
 1   customerId                786363 non-null  int64  
 2   creditLimit               786363 non-null  int64  
 3   availableMoney            786363 non-null  float64
 4   transactionDateTime       786363 non-null  object 
 5   transactionAmount         786363 non-null  float64
 6   merchantName              786363 non-null  object 
 7   acqCountry                786363 non-null  object 
 8   merchantCountryCode       786363 non-null  object 
 9   posEntryMode              786363 non-null  object 
 10  posConditionCode          786363 non-null  object 
 11  merchantCategoryCode      786363 non-null  object 
 12  currentExpDate            786363 non-null  object 
 13  accountOpenDate           786363 non-null  o

In [10]:
print(Counter(df["isFraud"]))

Counter({False: 773946, True: 12417})


In [17]:
pd.pandas.set_option('display.max_columns', None)

In [18]:
#Do not include both: accountNumber and customerId 
features = df.iloc[:,[1,2,3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27]] 
features.head()

,customerId,creditLimit,availableMoney,transactionAmount,merchantName,acqCountry,merchantCountryCode,posEntryMode,posConditionCode,merchantCategoryCode,currentExpDate,accountOpenDate,dateOfLastAddressChange,cardCVV,enteredCVV,cardLast4Digits,transactionType,echoBuffer,currentBalance,merchantCity,merchantState,merchantZip,cardPresent,posOnPremises,recurringAuthInd,expirationDateKeyInMatch
0,737265056,5000,5000.0,98.55,Uber,US,US,02,01,rideshare,06/2023,2015-03-14,2015-03-14,414,414,1803,PURCHASE,,0.0,,,,False,,,False
1,737265056,5000,5000.0,74.51,AMC #191138,US,US,09,01,entertainment,02/2024,2015-03-14,2015-03-14,486,486,767,PURCHASE,,0.0,,,,True,,,False
2,737265056,5000,5000.0,7.47,Play Store,US,US,09,01,mobileapps,08/2025,2015-03-14,2015-03-14,486,486,767,PURCHASE,,0.0,,,,False,,,False
3,737265056,5000,5000.0,7.47,Play Store,US,US,09,01,mobileapps,08/2025,2015-03-14,2015-03-14,486,486,767,PURCHASE,,0.0,,,,False,,,False
4,830329091,5000,5000.0,71.18,Tim Hortons #947751,US,US,02,01,fastfood,10/2029,2015-08-06,2015-08-06,885,885,3143,PURCHASE,,0.0,,,,True,,,False


In [19]:
labels = df['isFraud']
labels.head()

0    False
1    False
2    False
3    False
4    False
Name: isFraud, dtype: bool

In [20]:
#print the number of features in the dataset
print("Number of features/columns: ", features.shape[1]) 
#print the number of samples in the dataset
print("Number of samples/rows: ", features.shape[0]) 
#see useful summary statistics for numeric features
print(features.describe()) 

Number of features/columns:  26
Number of samples/rows:  786363
         customerId    creditLimit  availableMoney  transactionAmount  \
count  7.863630e+05  786363.000000   786363.000000      786363.000000   
mean   5.372326e+08   10759.464459     6250.725369         136.985791   
std    2.554211e+08   11636.174890     8880.783989         147.725569   
min    1.000881e+08     250.000000    -1005.630000           0.000000   
25%    3.301333e+08    5000.000000     1077.420000          33.650000   
50%    5.074561e+08    7500.000000     3184.860000          87.900000   
75%    7.676200e+08   15000.000000     7500.000000         191.480000   
max    9.993896e+08   50000.000000    50000.000000        2011.540000   

             cardCVV     enteredCVV  cardLast4Digits  currentBalance  
count  786363.000000  786363.000000    786363.000000   786363.000000  
mean      544.467338     544.183857      4757.417799     4508.739089  
std       261.524220     261.551254      2996.583810     6457.442

In [21]:
print("Number for rows in the predictor variable is", labels.shape[0])
labels.describe()

Number for rows in the predictor variable is 786363


count     786363
unique         2
top        False
freq      773946
Name: isFraud, dtype: object

In [25]:
#Split the data
#one-hot encoding for categorical variables
features = pd.get_dummies(features) 
#split the data into training and test data
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.33, random_state=42) 

In [26]:
print("Train shape:", features_train.shape, "Test shape:", features_test.shape)

Train shape: (526863, 6718) Test shape: (259500, 6718)


In [1]:
#standardize/normalize your numerical features
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

#use DataFrame.select_dtypes() to select float64 or int64 feature types automatically.
numerical_features = features.select_dtypes(include=['float64', 'int64'])
numerical_columns = numerical_features.columns # Get a list of the column names
ct = ColumnTransformer([("only numeric", StandardScaler(), numerical_columns)], remainder='passthrough')

#Fit your instance ct of ColumnTransformer to the training data and at the same time transform it by using the ColumnTransformer.fit_transform()
features_train_scaled = ct.fit_transform(features_train)
features_test_scaled = ct.fit_transform(features_test) #Transform your test data instance features_test

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


NameError: name 'features' is not defined

In [ ]:
print(features_train_scaled.describe())

In [ ]:
print(features_test_scaled.describe())

In [ ]:
#SMOTE ( handle unbalance isfraud catgeory )

In [ ]:
#design the model
model = Sequential()
model.add(InputLayer(input_shape=(features_train.shape[1],))) #input layer number of features 
model.add(Dense(28, activation='relu')) #hidden layer
model.add(Dense(2, activation='sigmoid')) #the output layer is a sigmoid with 2 units (0 or 1)

In [ ]:
# Compile the model.
model.compile(optimizer="adam",
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy(),
                       tf.keras.metrics.Recall(),
                       tf.keras.metrics.Precision(),
                      ])

In [ ]:
model.fit(
    features_train_scaled,
    labels_train,
    validation_split=.215,
    epochs=100,
    verbose=1,
)

In [ ]:
#get additional statistics
y_estimate = model.predict(features_train_scaled)
y_estimate = np.argmax(y_estimate, axis=1) ##using .argmax() convert the one-hot-encoded labels my_test_labels into the index of the class the sample belongs to

print(classification_report(y_true, y_estimate)) #.classification_report() method to calculate all the metrics.

In [ ]:
#sklearn.metrics.classification_report
y_estimate = model.predict(features_test_scaled)
y_true = model.argmax(label_test, axis=1) 
print(classification_report(y_true, y_estimate))

In [ ]:
hist = pd.DataFrame(model.history.history)

In [ ]:
hist.head()